In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
sparkSession = SparkSession.builder.config(conf=SparkConf() \
                                          .setAppName('Health Data Generator1') \
                                          .setMaster('local[4]')
                                          .set('spark.jars.packages','org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1')) \
                                    .enableHiveSupport() \
                                    .getOrCreate()

23/01/01 13:52:54 WARN Utils: Your hostname, audacious resolves to a loopback address: 127.0.0.2, but we couldn't find any external IP address!
23/01/01 13:52:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/audacious/spark-3.3.1-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/audacious/.ivy2/cache
The jars for the packages stored in: /home/audacious/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1a7628fc-b98e-4bbd-869d-18986da13e53;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

schema = StructType([StructField('Transaction_ID', StringType(), True) \
            , StructField('Transaction_Date', StringType(), True) \
            , StructField('Transaction_Value', IntegerType(), True) \
            , StructField('Transaction_Segment', StringType(), True) \
            , StructField('Credit_Card_ID', StringType(), True)])

df = sparkSession \
  .readStream \
  .format("kafka") \
  .option("checkpointLocation","/Spark_checkpointing_dir1/") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("subscribe", "health2") \
  .load()

df = df.select(from_json(col('value').cast("string"), schema).alias("data"))
data = df.select(col('data.Transaction_ID'), col('data.Transaction_Value'), col('data.Transaction_Segment'))
query = data \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

23/01/01 13:19:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d6dfe826-6de2-4464-8029-409237a4bc41. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/01 13:19:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+-----------------+-------------------+
|Transaction_ID|Transaction_Value|Transaction_Segment|
+--------------+-----------------+-------------------+
+--------------+-----------------+-------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID3047658494893|            31001|              SEG14|
|CTID3047658495685|             5702|              SEG15|
|CTID3047658496477|            17771|              SEG23|
|CTID3047658497270|             2545|              SEG22|
|CTID3047658498062|             9221|              SEG21|
|CTID3047658498854|            40742|              SEG25|
|CTID3047658499647|            10964|              SEG19|
|CTID3047658500439|            36626|              SEG23|
|CTID3047658501231|            40813|              SEG20|
|CTID3047658502023|            13704|              SEG14|
|CTID3047658502816|             4969|              SEG13|
|CTID3047658503608|            15428|              SEG16|
|CTID3047658504400|            25

-------------------------------------------
Batch: 2
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID3047658521830|            39172|              SEG14|
|CTID3047658522623|            44889|              SEG20|
|CTID3047658523415|             7395|              SEG13|
|CTID3047658524207|            35519|              SEG15|
|CTID3047658525000|            43428|              SEG20|
|CTID3047658525792|            36286|              SEG12|
|CTID3047658526584|            28453|              SEG22|
|CTID3047658527376|             6011|              SEG18|
|CTID3047658528169|            40127|              SEG15|
|CTID3047658528961|            21660|              SEG19|
|CTID3047658529753|             9058|              SEG23|
|CTID3047658530546|            42736|              SEG23|
|CTID3047658531338|            47

-------------------------------------------
Batch: 3
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID3200973546000|            42868|              SEG16|
|CTID3200973546792|            24926|              SEG15|
|CTID3200973547584|             8757|              SEG25|
|CTID3200973548376|            22848|              SEG20|
|CTID3200973549169|            35645|              SEG16|
+-----------------+-----------------+-------------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID3200973549961|            26381|              SEG25|
|CTID3200973550753|            41382|              

...

-------------------------------------------
Batch: 14
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID2930122440550|            24781|              SEG18|
|CTID2930122448473|            17860|              SEG21|
|CTID2930122456395|            26712|              SEG22|
+-----------------+-----------------+-------------------+

-------------------------------------------
Batch: 15
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID2930122464318|             5878|              SEG22|
|CTID2930122472241|            44493|              SEG14|
|CTID2930122480164|            24396|              SEG12|
|CTID2930122488087|            30334|            

-------------------------------------------
Batch: 16
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID3749317634814|            30849|              SEG15|
|CTID3749317642737|            26504|              SEG18|
|CTID3749317650660|            48292|              SEG11|
|CTID3749317658582|            18029|              SEG25|
|CTID3749317666505|            33278|              SEG24|
|CTID3749317674428|             5043|              SEG13|
|CTID3749317682351|             2219|              SEG13|
|CTID3749317690274|            21558|              SEG11|
|CTID3749317698197|             5846|              SEG22|
|CTID3749317706119|            16078|              SEG11|
|CTID3749317714042|            39557|              SEG15|
|CTID3749317721965|            44995|              SEG18|
+-----------------+-------------

-------------------------------------------
Batch: 22
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID6203927090910|            23766|              SEG23|
|CTID6203927098833|            26198|              SEG25|
|CTID6203927106756|            22741|              SEG16|
|CTID6203927114678|            33465|              SEG21|
|CTID6203927122601|            23028|              SEG13|
|CTID6203927130524|            27717|              SEG14|
|CTID6203927138447|            47713|              SEG17|
|CTID6203927146370|            37856|              SEG12|
|CTID6203927154293|            10357|              SEG23|
|CTID6203927162215|             2259|              SEG15|
|CTID6203927170138|            10253|              SEG13|
|CTID6203927178061|            18328|              SEG16|
|CTID6203927185984|             

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/audacious/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/audacious/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

schema = StructType([StructField('Transaction_ID', StringType(), True) \
            , StructField('Transaction_Date', StringType(), True) \
            , StructField('Transaction_Value', IntegerType(), True) \
            , StructField('Transaction_Segment', StringType(), True) \
            , StructField('Credit_Card_ID', StringType(), True)])

df = sparkSession \
  .readStream \
  .format("kafka") \
  .option("checkpointLocation","/Spark_checkpointing_dir1/") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("subscribe", "health2,health") \
  .load()

df = df.select(from_json(col('value').cast("string"), schema).alias("data"))
data = df.select(col('data.Transaction_ID'), col('data.Transaction_Value'), col('data.Transaction_Segment'))
query = data \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

23/01/01 13:24:15 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-59ef1db3-f251-4c48-8525-ea2e969a7122. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/01 13:24:15 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+-----------------+-------------------+
|Transaction_ID|Transaction_Value|Transaction_Segment|
+--------------+-----------------+-------------------+
+--------------+-----------------+-------------------+

-------------------------------------------
Batch: 41
-------------------------------------------
-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID1682145778556|             9872|              SEG21|
|CTID1682145779349|            46654|              SEG21|
|CTID1682145780141|             6270|              SEG12|
+-----------------+-----------------+-------------------+

+-----------------+-----------------+-------------------+
|   Transaction_ID|Tra

-------------------------------------------
Batch: 5
-------------------------------------------
-------------------------------------------
Batch: 45
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID1845257265193|            39315|              SEG13|
|CTID1845257265985|             3982|              SEG13|
|CTID1845257266777|            13443|              SEG11|
|CTID1845257267570|            13603|              SEG22|
|CTID1845257268362|            44852|              SEG20|
|CTID1845257269154|            22636|              SEG16|
|CTID1845257269947|             1271|              SEG13|
+-----------------+-----------------+-------------------+

+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/audacious/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/audacious/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

schema = StructType([StructField('Transaction_ID', StringType(), True) \
            , StructField('Transaction_Date', StringType(), True) \
            , StructField('Transaction_Value', IntegerType(), True) \
            , StructField('Transaction_Segment', StringType(), True) \
            , StructField('Credit_Card_ID', StringType(), True)])

df = sparkSession \
  .readStream \
  .format("kafka") \
  .option("checkpointLocation","/Spark_checkpointing_dir1/") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("subscribe", "health2,health") \
  .load()
df = df.select(from_json(col('value').cast("string"), schema).alias("data"))
data = df.select(col('data.Transaction_ID'), col('data.Transaction_Value'), col('data.Transaction_Segment'))

data = data.select(to_json(struct('*')).alias('value'))
query = data.writeStream \
  .format("kafka") \
  .option("checkpointLocation","/Spark_checkpointing_dir5/") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("topic", "health") \
  .start()
query.awaitTermination()

23/01/01 13:29:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 80
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID5606913445180|             9109|              SEG20|
|CTID5606913453103|            46310|              SEG18|
|CTID5606913461026|            32357|              SEG20|
+-----------------+-----------------+-------------------+

-------------------------------------------
Batch: 42
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID5606913445180|             9109|              SEG20|
|CTID5606913453103|            46310|              SEG18|
|CTID5606913461026|            32357|              SEG20|
+-----------------+-----------------+------------

-------------------------------------------
Batch: 48
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID5606913468948|            42626|              SEG23|
|CTID5606913476871|            26281|              SEG24|
|CTID5606913484794|            44528|              SEG16|
|CTID5606913492717|            38690|              SEG20|
|CTID5606913500640|             6073|              SEG21|
|CTID5606913508563|            34054|              SEG20|
|CTID5606913516485|            12774|              SEG17|
|CTID5606913524408|            18975|              SEG13|
|CTID5606913532331|            15826|              SEG14|
|CTID5606913540254|            45235|              SEG16|
|CTID5606913548177|            42992|              SEG11|
|CTID5606913556099|              389|              SEG12|
|CTID5606913564022|            4

-------------------------------------------
Batch: 54
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID5606913468948|            42626|              SEG23|
|CTID5606913476871|            26281|              SEG24|
|CTID5606913484794|            44528|              SEG16|
|CTID5606913492717|            38690|              SEG20|
|CTID5606913500640|             6073|              SEG21|
|CTID5606913508563|            34054|              SEG20|
|CTID5606913516485|            12774|              SEG17|
|CTID5606913524408|            18975|              SEG13|
|CTID5606913532331|            15826|              SEG14|
|CTID5606913540254|            45235|              SEG16|
|CTID5606913548177|            42992|              SEG11|
|CTID5606913556099|              389|              SEG12|
|CTID5606913564022|            4

-------------------------------------------
Batch: 60
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID5606913468948|            42626|              SEG23|
|CTID5606913476871|            26281|              SEG24|
|CTID5606913484794|            44528|              SEG16|
|CTID5606913492717|            38690|              SEG20|
|CTID5606913500640|             6073|              SEG21|
|CTID5606913508563|            34054|              SEG20|
|CTID5606913516485|            12774|              SEG17|
|CTID5606913524408|            18975|              SEG13|
|CTID5606913532331|            15826|              SEG14|
|CTID5606913540254|            45235|              SEG16|
|CTID5606913548177|            42992|              SEG11|
|CTID5606913556099|              389|              SEG12|
|CTID5606913564022|            4

...

-------------------------------------------
Batch: 343
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID5606913468948|            42626|              SEG23|
|CTID5606913476871|            26281|              SEG24|
|CTID5606913484794|            44528|              SEG16|
|CTID5606913492717|            38690|              SEG20|
|CTID5606913500640|             6073|              SEG21|
|CTID5606913508563|            34054|              SEG20|
|CTID5606913516485|            12774|              SEG17|
|CTID5606913524408|            18975|              SEG13|
|CTID5606913532331|            15826|              SEG14|
|CTID5606913540254|            45235|              SEG16|
|CTID5606913548177|            42992|              SEG11|
|CTID5606913556099|              389|              SEG12|
|CTID5606913564022|            

-------------------------------------------
Batch: 348
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID5606913468948|            42626|              SEG23|
|CTID5606913476871|            26281|              SEG24|
|CTID5606913484794|            44528|              SEG16|
|CTID5606913492717|            38690|              SEG20|
|CTID5606913500640|             6073|              SEG21|
|CTID5606913508563|            34054|              SEG20|
|CTID5606913516485|            12774|              SEG17|
|CTID5606913524408|            18975|              SEG13|
|CTID5606913532331|            15826|              SEG14|
|CTID5606913540254|            45235|              SEG16|
|CTID5606913548177|            42992|              SEG11|
|CTID5606913556099|              389|              SEG12|
|CTID5606913564022|            

-------------------------------------------
Batch: 354
-------------------------------------------
+-----------------+-----------------+-------------------+
|   Transaction_ID|Transaction_Value|Transaction_Segment|
+-----------------+-----------------+-------------------+
|CTID5606913468948|            42626|              SEG23|
|CTID5606913476871|            26281|              SEG24|
|CTID5606913484794|            44528|              SEG16|
|CTID5606913492717|            38690|              SEG20|
|CTID5606913500640|             6073|              SEG21|
|CTID5606913508563|            34054|              SEG20|
|CTID5606913516485|            12774|              SEG17|
|CTID5606913524408|            18975|              SEG13|
|CTID5606913532331|            15826|              SEG14|
|CTID5606913540254|            45235|              SEG16|
|CTID5606913548177|            42992|              SEG11|
|CTID5606913556099|              389|              SEG12|
|CTID5606913564022|            

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/audacious/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/audacious/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
schema = StructType([StructField('Transaction_ID', StringType(), True) \
            , StructField('Transaction_Date', StringType(), True) \
            , StructField('Transaction_Value', IntegerType(), True) \
            , StructField('Transaction_Segment', StringType(), True) \
            , StructField('Credit_Card_ID', StringType(), True)])


df = sparkSession.readStream.format('csv') \
            .schema(schema) \
            .option('cleanSource','archive') \
            .option('sourceArchiveDir', '/Spark_archive/') \
            .option("checkpointLocation","/Spark_checkpointing_dir/") \
            .option('latestFirst', True) \
            .option('spark.sql.streaming.forceDeleteTempCheckpointLocation',True) \
            .option('path','/Spark_streaming/TransactionBase/') \
            .load()

df = df.select(to_json(struct('*')).alias('value')).filter((df.Transaction_Value < 10000))
query = df.writeStream \
  .format("kafka") \
  .option("checkpointLocation","/Spark_checkpointing_dir/") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("topic", "health") \
  .start()
query.awaitTermination()

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = sparkSession \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("subscribe", "health2") \
  .load()

df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
query = df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [14]:
schema = StructType([StructField('Transaction_ID', StringType(), True) \
            , StructField('Transaction_Date', StringType(), True) \
            , StructField('Transaction_Value', StringType(), True) \
            , StructField('Transaction_Segment', StringType(), True) \
            , StructField('Credit_Card_ID', StringType(), True)])

df = sparkSession.read.format('kafka') \
            .option('kafka.bootstrap.servers','127.0.0.1:9092') \
            .option('subscribe','health2') \
            .load()
df.select(from_json(col('value').cast('String'), schema)).show()

+--------------------------------+
|from_json(CAST(value AS STRING))|
+--------------------------------+
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|            {null, null, null...|
|                            null|
|            {null, null, null...|
|            {null, null, null...|
+--------------------------------+
only showing top 20 rows



In [3]:
schema = StructType([StructField('Transaction_ID', StringType(), True) \
            , StructField('Transaction_Date', StringType(), True) \
            , StructField('Transaction_Value', IntegerType(), True) \
            , StructField('Transaction_Segment', StringType(), True) \
            , StructField('Credit_Card_ID', StringType(), True)])

df = sparkSession.read.format('kafka') \
            .option('kafka.bootstrap.servers','127.0.0.1:9092') \
            .option('subscribe','health2') \
            .load().select(from_json(col('value').cast('string'), schema= schema).alias('data'))
df = df.select('data.*')

In [4]:
df.show()

+--------------+----------------+-----------------+-------------------+--------------+
|Transaction_ID|Transaction_Date|Transaction_Value|Transaction_Segment|Credit_Card_ID|
+--------------+----------------+-----------------+-------------------+--------------+
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|          

In [7]:
schema = StructType([StructField('Transaction_ID', StringType(), True) \
            , StructField('Transaction_Date', StringType(), True) \
            , StructField('Transaction_Value', IntegerType(), True) \
            , StructField('Transaction_Segment', StringType(), True) \
            , StructField('Credit_Card_ID', StringType(), True)])

df = sparkSession.read.format('kafka') \
            .option('kafka.bootstrap.servers','127.0.0.1:9092') \
            .option('subscribe','health2') \
            .option("startingOffsets", "earliest") \
              .option("endingOffsets", "latest") \
            .load().select(from_json(col('value').cast('string'), schema= schema).alias('data'))
df = df.select('data.*')


In [8]:
df.show()

+--------------+----------------+-----------------+-------------------+--------------+
|Transaction_ID|Transaction_Date|Transaction_Value|Transaction_Segment|Credit_Card_ID|
+--------------+----------------+-----------------+-------------------+--------------+
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|             null|               null|          null|
|          null|            null|          